In [1]:
pip install ffprobe

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install ffmpeg

Note: you may need to restart the kernel to use updated packages.


In [3]:
# code copied from the labs in order to add path variable
# and run ffmpeg locally
exist = !which ffmpeg
if not exist:
  !brew https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz -o ffmpeg.tar.xz \
     && tar -xf ffmpeg.tar.xz && rm ffmpeg.tar.xz
  ffmdir = !find . -iname ffmpeg-*-static
  path = %env PATH
  path = path + ':' + ffmdir[0]
  %env PATH $path

!which ffmpeg

/usr/local/bin/ffmpeg


In [4]:
import ffmpeg
import os

In [5]:
# function to extract everything after the equal sign of a string
def extract_info(data):
    #initialize an empty string
    temp = ""
    # set a check variable to false until the equal sign has
    # been reached
    check = False
    for i in range(len(data)):
        # once the equal sign has been reached, append all remaining 
        # characters to the temp string
        if check == True:
            temp = temp + data[i]
        # once the for loop reaches the equal sign, set check variable 
        # equal to true
        if data[i] == "=":
            check = True
    return temp

In [6]:
from os import walk

In [7]:
file_names = os.listdir("Exercise3_Films")

In [8]:
# function to determine which aspects of the digital file format
# doesn't meet the required specifications and print to .txt file which 
# fields are problematic
def check_parameters(path):
    # save video data in a list
    video_data = !ffprobe -v error -select_streams v:0 -show_entries stream=codec_name,r_frame_rate,display_aspect_ratio,width,height,bit_rate -of default=noprint_wrappers=1 $path
    # save audio data in a list
    audio_data = !ffprobe -v 0 -select_streams a:0 -show_entries stream=codec_name,bit_rate,channel_layout -of default=noprint_wrappers=1 $path
    # save string values of the relevant video/audio data
    vcodec = extract_info(video_data[0])
    width = extract_info(video_data[1])
    height = extract_info(video_data[2])
    dar = extract_info(video_data[3])
    frame_rate = extract_info(video_data[4])
    v_bit_rate = extract_info(video_data[5])
    acodec = extract_info(audio_data[0])
    channel = extract_info(audio_data[1])
    a_bit_rate = extract_info(audio_data[2])
    
    # open .txt file to write to
    f = open("report-file.txt", "a")
    # print which file is currently being reported about
    f.write(path + "\n")
    
    # check if last four digits meet specified file type of .mp4
    file_type = ""
    for i in range(len(path) - 4, len(path)):
        file_type += path[i]
    
    # set check variable to true until some specification isn't met
    format_ok = True
    if file_type != ".mp4":
        f.write("Wrong file type.\n")
        format_ok = False
    # check if video codec is correct
    if vcodec != "h264":
        f.write("Video codec is incorrectly formatted.\n")
        format_ok = False
    # check if width is correct
    if width != "640":
        f.write("Width is incorrectly formatted.\n")
        format_ok = False
    # check if height is correct
    if height != "360":
        f.write("Height is incorrectly formatted.\n")
        format_ok = False
    # check if aspect ratio is correct
    if dar != "16:9":
        f.write("Aspect ratio is incorrectly formatted.\n")
        format_ok = False
    # check if frame rate is correct
    if frame_rate != "25/1":
        f.write("Frame rate is incorrectly formatted.\n")
        format_ok = False
    # check if video bit rate is within correct range
    if int(v_bit_rate) < 2000000 or int(v_bit_rate) > 5000000:
        f.write("Video bit rate is incorrectly formatted.\n")
        format_ok = False
    # check if audio codec is correct
    if acodec != "aac":
        f.write("Audio codec is incorrectly formatted.\n")
        format_ok = False
    # check if audio channels is correct
    if channel != "stereo":
        f.write("Audio channels is incorrectly formatted.\n")
        format_ok = False
    # check if audio bit rate is correct
    if int(a_bit_rate) > 256000:
        f.write("Audio bit rate is incorrectly formatted.\n")
        format_ok = False
    # if file format is incorrect, update to correct file specifications
    if format_ok == False:
        temp = path.split(".")[0]
        temp = temp + "_formatOK.mp4"
        !ffmpeg -hide_banner -y -i $path -vcodec h264 -vf scale=640:360,setdar=16/9 -r 25 -b:v 3.5M -b:a 128k -ac 2 -acodec aac $temp
    # if file is formatted correctly, don't do anything
    else:
        f.write("File is correctly formatted.\n")
    f.write("-------------------------------------------\n")
    f.close()

In [9]:
# go through list of file paths and check for required specifications
for i in range(1, len(file_names)):
    temp = "Exercise3_Films/"
    temp = temp + file_names[i]
    check_parameters(temp)

9285191
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'Exercise3_Films/Last_man_on_earth_1964.mov':
  Metadata:
    major_brand     : qt  
    minor_version   : 537199360
    compatible_brands: qt  
    creation_time   : 2021-08-02T18:38:19.000000Z
  Duration: 00:00:20.02, start: 0.000000, bitrate: 11240 kb/s
  Stream #0:0[0x1](eng): Video: prores (LT) (apcs / 0x73637061), yuv422p10le(tv, smpte170m/smpte170m/bt709, progressive), 640x360, 9285 kb/s, SAR 1:1 DAR 16:9, 23.98 fps, 23.98 tbr, 24k tbn (default)
    Metadata:
      creation_time   : 2021-08-02T18:38:19.000000Z
      handler_name    : Apple Video Media Handler
      vendor_id       : appl
      encoder         : Apple ProRes 422 LT
      timecode        : 00:51:43:11
  Stream #0:1[0x2](eng): Audio: pcm_s16le (sowt / 0x74776F73), 48000 Hz, stereo, s16, 1536 kb/s (default)
    Metadata:
      creation_time   : 2021-08-02T18:38:19.000000Z
      handler_name    : Apple Sound Media Handler
      vendor_id       :     
      timecode     

frame=  502 fps=140 q=-1.0 Lsize=    8567kB time=00:00:20.05 bitrate=3499.6kbits/s dup=0 drop=98 speed=5.58x    
video:8236kB audio:314kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.191515%
[libx264 @ 0x7fa027f08040] frame I:5     Avg QP: 4.35  size: 54210
[libx264 @ 0x7fa027f08040] frame P:129   Avg QP: 6.87  size: 37670
[libx264 @ 0x7fa027f08040] frame B:368   Avg QP:10.40  size:  8975
[libx264 @ 0x7fa027f08040] consecutive B-frames:  1.6%  0.4%  4.8% 93.2%
[libx264 @ 0x7fa027f08040] mb I  I16..4: 19.2% 33.7% 47.1%
[libx264 @ 0x7fa027f08040] mb P  I16..4:  4.0% 18.8% 11.3%  P16..4: 17.3% 20.8% 17.6%  0.0%  0.0%    skip:10.1%
[libx264 @ 0x7fa027f08040] mb B  I16..4:  0.1%  1.0%  0.6%  B16..8: 37.9% 19.1% 11.6%  direct: 7.7%  skip:21.9%  L0:46.1% L1:38.9% BI:14.9%
[libx264 @ 0x7fa027f08040] final ratefactor: 7.81
[libx264 @ 0x7fa027f08040] 8x8 transform intra:53.3% inter:42.0%
[libx264 @ 0x7fa027f08040] coded y,uvDC,uvAC intra: 87.3% 95.2% 95.0% inter: 38.5% 59

frame=  502 fps=127 q=-1.0 Lsize=    8841kB time=00:00:20.03 bitrate=3615.4kbits/s dup=0 drop=98 speed=5.08x    
video:8507kB audio:317kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.185621%
[libx264 @ 0x7ff5fd809240] frame I:3     Avg QP: 8.17  size: 46004
[libx264 @ 0x7ff5fd809240] frame P:127   Avg QP: 9.10  size: 35909
[libx264 @ 0x7ff5fd809240] frame B:372   Avg QP:12.07  size: 10786
[libx264 @ 0x7ff5fd809240] consecutive B-frames:  1.2%  0.0%  0.0% 98.8%
[libx264 @ 0x7ff5fd809240] mb I  I16..4: 11.7% 59.0% 29.3%
[libx264 @ 0x7ff5fd809240] mb P  I16..4:  4.6% 29.4%  9.8%  P16..4: 13.3% 24.4% 17.7%  0.0%  0.0%    skip: 0.7%
[libx264 @ 0x7ff5fd809240] mb B  I16..4:  0.9%  4.0%  1.2%  B16..8: 30.9% 28.3% 13.9%  direct:11.8%  skip: 9.0%  L0:42.1% L1:37.7% BI:20.1%
[libx264 @ 0x7ff5fd809240] final ratefactor: 9.14
[libx264 @ 0x7ff5fd809240] 8x8 transform intra:66.4% inter:42.4%
[libx264 @ 0x7ff5fd809240] coded y,uvDC,uvAC intra: 90.6% 98.0% 97.5% inter: 54.7% 65